# Segmentation model - Version 1

The classification backbone of the model is VGG-16 network

In [ ]:
import os
import keras
import tensorflow as tf
import keras.backend as K

## Building model

In [ ]:

def build_model(optimizer, learning_rate, out_type):

    policy = tf.keras.mixed_precision.experimental.Policy('mixed_float16')#sets values to be float16 for nvidia 2000,3000 series GPUs, plus others im sure
    
    input_img = keras.layers.Input(shape=(512, 512, 1))
    x = keras.layers.Lambda(lambda x: K.cast_to_floatx(x))(input_img)
    x = keras.layers.Lambda(lambda x: x/255.)(x)
    
    x = keras.layers.BatchNorm()(x)
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(8, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNorm()(x)

    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNorm()(x)
    
    x = keras.layers.Conv2D(16, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(16, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNorm()(x)
    
    x = keras.layers.Conv2D(32, 2,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.Conv2D(32, 3,padding='valid',activation='selu', kernel_initializer='lecun_normal',dtype=policy)(x)
    x = keras.layers.MaxPooling2D(2)(x)
    x = keras.layers.BatchNorm()(x)    

    x = keras.layers.Conv2D(16, 1, activation="selu",kernel_initializer="lecun_normal",dtype=policy)(x)
    x = keras.layers.Conv2D(1, 1, activation='linear')(x)
    x_map = keras.activations.Activation("sigmoid")(x)
    

    
    y = keras.layers.GlobalMaxPooling2D()(x_map)
    
    if opt == "SGD":
        #looks like pretty good but noisy results with no momentum, lets check 0.9...
        optimizer = keras.optimizers.SGD(lr=lr)
    if opt == "RMSprop":
        optimizer = keras.optimizers.RMSprop(learning_rate=lr)
    if opt == "Adam":
        optimizer = keras.optimizers.Adam(learning_rate=lr)

    if out_type == "classify":
        model = keras.models.Model(inputs=input_img, outputs=y)
    if out_type == "map":
        model = keras.models.Model(inputs=input_img, outputs=x_map)
    model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy','AUC'])

    return model



In [ ]:
map_model = build_model("SGD", 1e-2, "map")
model = build_model("SGD", 1e-2, "classify")
model.summary()

## CASTING DATA

### Pre-processing


### Training model

### Testing the model

## DEEP PCB DATA

### Pre-processing

### Training model

## MAGNETIC TILE DATA

### Pre-processing

### Training model

## ROAD CRACK DATA

### Pre-processing

### Training model

## SOLAR PANEL DATA

### Pre-processing

### Training the model

## STEEL DEFECT DATA

### Pre-processing

### Training the model

## SURFACE DEFECT DATA

### Pre-processing

### Training the model

## WELDING DATA

### Pre-processing

### Training model